In [1]:
import pyodbc
import sqlite3
import os.path
import sys
import pprint
sys.path.append(r'/home/ernst/Projects/Prometheon/rest-api-azure')
# from numpy import product as npproduct
# from numpy import array as nparray
from datetime import datetime
from numpy import product as npproduct
from numpy import array as nparray
from project_config.config import (sql_parameter_tablename_dict,
                                  required_keys,
                                  base_rate,
                                  db_path,
                                  AZURE_SERVER,
                                  AZURE_USERNAME,
                                  AZURE_PASSWORD,
                                  AZURE_DATABASE,
                                  AZURE_PORT)

In [2]:
def query_azure(sql, values):
    driver= '{ODBC Driver 13 for SQL Server}'
    cnxn = pyodbc.connect((r"DRIVER={driver};PORT={port};SERVER={server};"
                           r"DATABASE={database};UID={username};PWD={password}").format(driver=driver,
                                                    port=AZURE_PORT,
                                                    server=AZURE_SERVER,
                                                    database=AZURE_DATABASE,
                                                    username=AZURE_USERNAME,
                                                    password=AZURE_PASSWORD))
    cursor = cnxn.cursor()
    try:
        # cursor.execute("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE='BASE TABLE'")
        cursor.execute(sql, values)
        rows = cursor.fetchall()
        columns = [column[0] for column in cursor.description]
        return(columns, rows)
    except Exception as e:
        print(e)
    finally: # Always executed, even in case of exception
        cnxn.close()
        

In [3]:
sql1 = 'SELECT * FROM [dbo].[Pricing_Param_Ref_Data] WHERE Reference_Data_ID > 1200 and  Reference_Data_ID < 1300 and Product_ID = 1'
sql2 = 'SELECT * FROM [dbo].[Pricing_Param_Ref_Data] WHERE Reference_Data_ID = 577 and Product_ID = 2'
sql3 = 'SELECT * FROM [dbo].[Pricing_Param_Ref_Data] WHERE Reference_Data_ID = ? and Product_ID = ?'

In [4]:
sql_unique = 'SELECT COUNT(1), Reference_Data_Type FROM [dbo].[Pricing_Param_Ref_Data] GROUP BY Reference_Data_Type'


In [11]:
sql = """SELECT * FROM [dbo].[Pricing_Param_General] WHERE Product_ID = ?
                AND Pricing_Parameter_Name = ? AND Valid_From <= ? AND Valid_To > ?"""
vals = (1, 'Base factor', datetime(2018, 1, 28, 0, 0), datetime(2018, 1, 28, 0, 0))

In [24]:
sql = """SELECT * FROM [dbo].[Pricing_Param_General] WHERE Product_ID = ? AND Pricing_Parameter_Name = ? 
                """
vals = (1, ' Base factor')

In [25]:
columns, row = query_azure(sql, vals)

In [26]:
columns

['Pricing_Parameter_Value',
 'Product_ID',
 'Product_Description',
 'Pricing_Parameter_Type_ID',
 'Pricing_Parameter_Type_Name',
 'Pricing_Parameter_ID',
 'Pricing_Parameter_Name',
 'Valid_From',
 'Valid_To']

In [27]:
row

[(50.0, 1, 'Temperature only', 8, 'General Pricing Parameter', 45, ' Base factor', datetime.date(2018, 1, 1), datetime.date(9999, 12, 31))]

In [9]:
pricing_parameter_ref_keys = ['start_country',
                                      'end_country',
                                      'airport', 
                                      'container_type',
                                      'start_trucking_time',
                                      'end_trucking_time',
                                      'temperature_range'
                                      ]
pricing_parameter_bus_keys = ['forwarder',
                             'ground_handler']

In [ ]:
sql_parameter_tablename_dict.keys()

In [ ]:
premium_request= {
  "sum_insured" : 360000,
  "product_id" : 1,
  "temperature_range" : 1405,
  "container_type": 1151,
  "forwarder" : 91,
  "start_trucking_time" : 1457,
  "start_country" : 1266,
  "airport" : "580, 647, 712",
  "ground_handler" : "10, 25, 58",
  "end_trucking_time" : 1455,
  "end_country" : 1293,
  "timestamp":"2018-01-28"
}

In [ ]:
def query_azure(sql, vals, fetch_one=True):
    # Make database connection
    db_connection = pyodbc.connect((r"DRIVER={driver};PORT={port};SERVER={server};"
                           r"DATABASE={database};UID={username};PWD={password}").format(driver=driver,
                                                    port=AZURE_PORT,
                                                    server=AZURE_SERVER,
                                                    database=AZURE_DATABASE,
                                                    username=AZURE_USERNAME,
                                                    password=AZURE_PASSWORD))
    cursor = db_connection.cursor()
    try:
        cursor.execute(sql, vals)
        columns = [column[0] for column in cursor.description]   
        if fetch_one:
            row = cursor.fetchone()    
        else:
            row = cursor.fetchall()
        return columns, row
    except Exception as e:
        print(e)
    finally:
        db_connection.close() 


In [ ]:
driver= '{ODBC Driver 13 for SQL Server}'

def multi_query_azure(sql_list, vals_list, fetch_one=True):
    # Make database connection
    db_connection = pyodbc.connect((r"DRIVER={driver};PORT={port};SERVER={server};"
                           r"DATABASE={database};UID={username};PWD={password}").format(driver=driver,
                                                    port=AZURE_PORT,
                                                    server=AZURE_SERVER,
                                                    database=AZURE_DATABASE,
                                                    username=AZURE_USERNAME,
                                                    password=AZURE_PASSWORD))
    cursor = db_connection.cursor()
    try:
        for sql, vals in zip(sql_list, vals_list):
            cursor.execute(sql, vals)
            columns = [column[0] for column in cursor.description]          
            if fetch_one:
                row = cursor.fetchone()    
            else:
                row = cursor.fetchall()
            yield (columns, row)
    except Exception as e:
        print('Error in multi_query_azure: \n')
        print(e)
    finally:
        db_connection.close() 

In [ ]:
## NEW


## Todo's:
# + filter on timestamp
# + get "other_parameters" (administration, fees, ..)
# + send sql and vals in a list, to open the db_connection only once, and execute the cursor many times
def calculate_premium(premium_request):
    """
    premium_request : a dictionary
    
    The contents of the request can be divided into two groups:
    1. elements that result in one parameter per element (multiplier for the premium)
      (airport, container_type, start_country, ....)
    2. general elements that in another way determine the premium or parameter values
      (sum_insured, timestamp, temperature_range, product_id)
    
    The first group of elements can be subdivided into:
    1.  a)  business partners.
            (forwarder, ground_handler)
        b)  other pricing parameters
            (airport, container_type, start_country, end_country, temperature_range,
            start_trucking_time, end_trucking_time)
    
    Other pricing parameters depend only on product_id
       (administration, taxes, fees, Base factor)
       
    This function calculates the premium using the following steps:
       - data transformations (comma-separated strings to list, string to date)
       - retrieving the elements in 2. (sum_insured, timestamp, ... etc)
       - iterate over the elements of 1.b to retrieve the parameters from the DB, and append parameters to list
       - iterate over the elements of 1.a to retrieve the parameters from the DB, and append parameters to list
       - iterate over the other_pricing_params (administration, fees, ) to retrieve those pricing parameters, 
         and append to another list
       - calculate the premium using all collected parameters 
       - depending on the request, return only the premium or the premium and the details 
         (or a message, in case of error)
 
    """
    # 0. Initializing/setting variables
    csv_list_keys = ['airport', 'ground_handler'] # keys for items that need unpacking
    include_details = True if ('details' in premium_request) and \
        (premium_request['details'] == 1) else False

    # column names for the Pricing_Param_Ref_Data table
    param_colnames = ['Pricing_Parameter_Value', 'Reference_Data_Name', 
                        'Pricing_Parameter_Type_Name', 'Valid_From', 'Valid_To'] 
    
    # column names for the Pricing_Param_Bus_Partner table
    bus_colnames = ['Pricing_Parameter_Value', 'Business_Partner_Name', 
                        'Pricing_Parameter_Type_Name', 'Valid_From', 'Valid_To'] 
    
    general_colnames = ['Pricing_Parameter_Value', 'Pricing_Parameter_Name',
                        'Pricing_Parameter_Type_Name', 'Valid_From', 'Valid_To'] 
    # general parameters
    general_params = ['Loss adjustment expense',
                     'Administration (fixed)',
                     'Administration (variable)',
                     'Taxes',
                     'Fees',
                     'Target profit',
                     'Base factor'] 
    # 1. Data transformation and retrieving elements

    # split comma-separated strings, put into list
    for key in csv_list_keys:
        premium_request[key] = [int(r) for r in
            premium_request[key].split(',')]

    # timestamp. input is a string: 'yyyy-mm-dd'. do datetime
    timestamp = datetime.strptime(premium_request['timestamp'], '%Y-%m-%d')
    
    # to string (for reply)
    timestamp_str = datetime.strftime(timestamp, '%Y-%m-%d')
        
    # product_id and amount 
    product_id = premium_request['product_id']
    sum_insured = premium_request['sum_insured']  
    
    # put into dictionary
    quote_reply = {'sum_insured': sum_insured,
                   'product_id': product_id,
                   'timestamp': timestamp_str,
                    'parameter_list': [],
                  'general_params':{}}       
    # 2. Iterate over elements of 1.b and 1.a to gather the "parameters"
    # (premium multipliers)
    # Loop through all items, append the sql-queries and the values to a list.
    # Those will be sent off all at once (to speed-up retrieval)
    sql_list, vals_list = [], []
    for key, parameter_ids in premium_request.items():
        
        # Make sure we can iterate over the parameter id's 
        # (if list, fine, else: make single-item list)
        if not type(parameter_ids) == list:
            parameter_ids = [parameter_ids]
        
        # Test if this is a key that requires a parameter lookup
        # if not, skip it
        if key in pricing_parameter_ref_keys:
            sql = """ SELECT * FROM [dbo].[Pricing_Param_Ref_Data] WHERE 
                    Reference_Data_ID = ? AND Product_ID = ? AND 
                    Valid_From <= ? AND Valid_To > ?
                  """

        elif key in pricing_parameter_bus_keys:
            sql = """ SELECT * FROM [dbo].[Pricing_Param_Bus_Partner] WHERE 
                    Business_Partner_ID = ? AND Product_ID = ? AND 
                    Valid_From <= ? AND Valid_To > ?
                  """            

        # when the key is one of the parameter keys, do the queries for the parameters
        if (key in pricing_parameter_ref_keys) or (key in pricing_parameter_bus_keys): 
            
            for parameter_id in parameter_ids:
                vals = (parameter_id, product_id, timestamp, timestamp) #Timestamp not used yet!
                sql_list.append(sql)
                vals_list.append(vals)
        else:
            continue # for clarity  
    
    # Send off the queries that are stored in the lists to the SQL server        
    for (columns, row) in multi_query_azure(sql_list, vals_list):

        # find the index of the row where the Pricing Parameter Value etc. is located
        # NB: not pretty, too much repetition. Fix at some time when there is time. 
        try:
            pricing_param_value_index = [i for i, col_name in enumerate(columns) 
                               if col_name ==  param_colnames[0]][0]
            ref_data_name_index = [i for i, col_name in enumerate(columns) 
                               if col_name == param_colnames[1]][0]
            name_index = [i for i, col_name in enumerate(columns) 
                               if col_name == param_colnames[2]][0] 
            valid_from_index = [i for i, col_name in enumerate(columns) 
                               if col_name == param_colnames[3]][0] 
            valid_to_index = [i for i, col_name in enumerate(columns) 
                               if col_name == param_colnames[4]][0]             
        except:
            pricing_param_value_index = [i for i, col_name in enumerate(columns) 
                               if col_name == bus_colnames[0]][0]
            ref_data_name_index = [i for i, col_name in enumerate(columns) 
                               if col_name == bus_colnames[1]][0]
            name_index = [i for i, col_name in enumerate(columns) 
                               if col_name == bus_colnames[2]][0]                 
            valid_from_index = [i for i, col_name in enumerate(columns) 
                               if col_name == bus_colnames[3]][0] 
            valid_to_index = [i for i, col_name in enumerate(columns) 
                               if col_name == bus_colnames[4]][0] 
            
        # Get the values
        pricing_param_value = row[pricing_param_value_index]  
        ref_data_name = row[ref_data_name_index]  
        pricing_param_type_name = row[name_index] 
        valid_from = row[valid_from_index]
        valid_to = row[valid_to_index] 

    # Append to the list

        quote_reply['parameter_list'].append({'type': key,
                            'pricing_param_type_name': pricing_param_type_name,
                            'pricing_param_value': pricing_param_value,
                             'ref_data_name': ref_data_name,
                              'valid_from': datetime.strftime(valid_from, '%Y-%m-%d'),
                              'valid_to': datetime.strftime(valid_to, '%Y-%m-%d')})
            
    ## 3. Get parameters from the Pricing_Param_General table
    # Also here, create the SQL statements in a list, and use the generator
    # to iterate through the results
    # general_params are stored as a dict within quote_reply
    sql_list = []
    vals_list = []
    for general_param in general_params:
        sql_list.append("""SELECT * FROM [dbo].[Pricing_Param_General] WHERE Product_ID = ?
                AND Pricing_Parameter_Name = ? AND Valid_From <= ? AND Valid_To > ? 
              """)
        vals_list.append((product_id, general_param, timestamp, timestamp))
        

    for columns, row in multi_query_azure(sql_list, vals_list):
        pricing_param_value_index = [i for i, col_name in enumerate(columns) 
                           if col_name == general_colnames[0]][0]
        name_index = [i for i, col_name in enumerate(columns) 
                           if col_name == general_colnames[1]][0]
        type_name_index = [i for i, col_name in enumerate(columns) 
                           if col_name == general_colnames[2]][0]
        valid_from_index = [i for i, col_name in enumerate(columns) 
                           if col_name == general_colnames[3]][0] 
        valid_to_index = [i for i, col_name in enumerate(columns) 
                           if col_name == general_colnames[4]][0] 

        pricing_param_value = row[pricing_param_value_index]  
        pricing_parameter_name = row[name_index]
        pricing_param_type_name = row[type_name_index]  
        valid_from = row[valid_from_index]
        valid_to = row[valid_to_index] 
        quote_reply['general_params'][pricing_parameter_name] = {'pricing_param_value': pricing_param_value,
                                                            'valid_from': datetime.strftime(valid_from, '%Y-%m-%d'),
                                                            'valid_to': datetime.strftime(valid_to, '%Y-%m-%d')}
        
    # Process the obtained data to calculate the premium
    parameters = [item['pricing_param_value'] for item in quote_reply['parameter_list'] 
                  if 'pricing_param_value' in item.keys() ]

    # Multiply all parameters
    parameters_product = 1
    for par in parameters:
        parameters_product *= par
        
    # NB: the expression below is not right. Especially the base factor (no idea what to do with 50?)
    premium = premium_request['sum_insured'] * (
                quote_reply['general_params']['Base factor']['pricing_param_value']/1000 *
                (1 + quote_reply['general_params']['Administration (variable)']['pricing_param_value'] +
                quote_reply['general_params']['Fees']['pricing_param_value']) *
                parameters_product) + (
                quote_reply['general_params']['Administration (fixed)']['pricing_param_value'])
    if include_details:
        return (premium, quote_reply)
    else:
        return premium

In [ ]:
premium_request= {
  "sum_insured" : 360000,
  "product_id" : 1,
  "temperature_range" : 1405,
  "container_type": 1151,
  "forwarder" : 91,
  "start_trucking_time" : 1457,
  "start_country" : 1266,
  "airport" : "580, 647, 712",
  "ground_handler" : "10, 25, 58",
  "end_trucking_time" : 1455,
  "end_country" : 1293,
  "timestamp":"2018-01-28",
   "details": 1}
reply = calculate_premium(premium_request)
try:
    premium, quote_reply = reply
except TypeError as e:
    # If there is only a single return item, an error has occurred
    # Then, the tuple unpacking will fail (with a TypeError)
    # The error is communicated by a message
    message = reply
    


In [ ]:
pprint.pprint(reply)


In [ ]:


driver= '{ODBC Driver 13 for SQL Server}'
csv_list_keys = ['airport', 'ground_handler'] # keys for items that need unpacking

## Todo's:
# - filter on timestamp
# - get "other_parameters" (administration, fees, ..)
# - send sql and vals in a list, to open the db_connection only once, and execute the cursor many times
def calculate_premium(premium_request):
    """
    premium_request : a dictionary
    
    The contents of the request can be divided into two groups:
    1. elements that result in one parameter per element (multiplier for the premium)
      (airport, container_type, start_country, ....)
    2. general elements that in another way determine the premium or parameter values
      (sum_insured, timestamp, temperature_range, product_id)
    
    The first group of elements can be subdivided into:
    1.  a)  business partners.
            (forwarder, ground_handler)
        b)  other pricing parameters
            (airport, container_type, start_country, end_country, temperature_range,
            start_trucking_time, end_trucking_time)
    
    Other pricing parameters depend only on product_id
       (administration, taxes, fees, Base factor)
       
    This function calculates the premium using the following steps:
       - data transformations (comma-separated strings to list, string to date)
       - retrieving the elements in 2. (sum_insured, timestamp, ... etc)
       - iterate over the elements of 1.b to retrieve the parameters from the DB, and append parameters to list
       - iterate over the elements of 1.a to retrieve the parameters from the DB, and append parameters to list
       - iterate over the other_pricing_params (administration, fees, ) to retrieve those pricing parameters, 
         and append to another list
       - calculate the premium using all collected parameters 
       - depending on the request, return only the premium or the premium and the details 
         (or a message, in case of error)
 
    """
    # 0. Initializing/setting variables
    # init the list that contains the elements that determine the premium
 
    # 1. Data transformation and retrieving elements

    # split comma-separated strings, put into list
    for key in csv_list_keys:
        premium_request[key] = [int(r) for r in
            premium_request[key].split(',')]

    # timestamp. input is a string: 'yyyy-mm-dd'. do datetime
    timestamp = datetime.strptime(premium_request['timestamp'], '%Y-%m-%d')
    
    # to string (for reply)
    timestamp_str = datetime.strftime(timestamp, '%Y-%m-%d')
        
    # product_id and amount 
    product_id = premium_request['product_id']
    sum_insured = premium_request['sum_insured']  
    
    # put into dictionary
    quote_reply = {'sum_insured': sum_insured,
                   'product_id': product_id,
                   'timestamp': timestamp_str,
                    'parameter_list': []}       
    # 2. Iterate over elements of 1.b and 1.a
    # Loop through all items, to search for the "parameters"/premium multipliers
    for key, parameter_ids in premium_request.items():
        
        # Make sure we can iterate over the parameter id's 
        # (if list, fine, else: make single-item list)
        if not type(parameter_ids) == list:
            parameter_ids = [parameter_ids]
        
        # Test if this is a key that requires a parameter lookup
        # if not, skip it
        if key in pricing_parameter_ref_keys:
            sql = """ SELECT * FROM [dbo].[Pricing_Param_Ref_Data] WHERE 
                    Reference_Data_ID = ? and Product_ID = ?
                  """
            colnames = ['Pricing_Parameter_Value', 'Reference_Data_Name', 
                        'Pricing_Parameter_Type_Name']
        elif key in pricing_parameter_bus_keys:
            sql = """ SELECT * FROM [dbo].[Pricing_Param_Bus_Partner] WHERE 
                    Business_Partner_ID = ? and Product_ID = ?
                  """            
            colnames = ['Pricing_Parameter_Value', 'Business_Partner_Name', 
                        'Pricing_Parameter_Type_Name']
        # when the key is one of the parameter keys, do the queries for the parameters
        if (key in pricing_parameter_ref_keys) or (key in pricing_parameter_bus_keys): 
            
            for parameter_id in parameter_ids:
                vals = (parameter_id, product_id) #Timestamp not used yet!
                columns, row = query_azure(sql, vals)
                # print(columns)

                # find the index of the row where the Pricing Parameter Value is located
                # NB: those strings should probably be in config
                pricing_param_value_index = [i for i, col_name in enumerate(columns) 
                                   if col_name ==  colnames[0]][0]
                ref_data_name_index = [i for i, col_name in enumerate(columns) 
                                   if col_name == colnames[1]][0]
                name_index = [i for i, col_name in enumerate(columns) 
                                   if col_name == colnames[2]][0] 

                # Get the values
                pricing_param_value = row[pricing_param_value_index]  
                ref_data_name = row[ref_data_name_index]  
                pricing_param_type_name = row[name_index] 

                # Append to the list

                quote_reply['parameter_list'].append({'type': key,
                                    'pricing_param_type_name': pricing_param_type_name,
                                    'pricing_param_value': pricing_param_value,
                                    'ref_data_name': ref_data_name})

            
        else:
            continue # for clarity
    

    # Process the obtained data to calculate the premium
    parameters = [item['pricing_param_value'] for item in quote_reply['parameter_list'] 
                  if 'pricing_param_value' in item.keys() ]
    # Multiply all parameters
    parameters_product = 1
    for par in parameters:
        parameters_product *= par
    premium = (premium_request['sum_insured'] * base_rate * parameters_product)
    return (premium, quote_reply)

In [ ]:
a = tf()

In [ ]:
rr, rrr = a

In [ ]:
rrr

In [ ]:
for key, parameter_ids in premium_request.items():
    print(key, parameter_ids)

In [ ]:
p_0 =1
for par in parameters:
    p_0 *= par

In [ ]:
p_0


In [ ]:
print(premium)
print(quote_reply)

(1351, 2, 1.8, 'Freight insurance', 5, 'Country', 14, 'Medium risk', 'Country', 'Saudi Arabia'),

(1351, 1, 1.5, 'Temperature only', 5, 'Country', 14, 'Medium risk', 'Country', 'Saudi Arabia')
